In [1]:
%matplotlib widget

In [2]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df_typhoon = pd.read_csv('TRAN_COMPLEX.csv',index_col=False)
data = sio.matlab.loadmat('ctrl_cplx')

In [4]:
def abc2dq(times,a,b,c,omega=2*np.pi*50,theta_0=0.0):

    Dt = times[1]-times[0] 
    d = times*0.0
    q = times*0.0
    for it in range(len(times)):

        theta = Dt*it*omega + theta_0
        abc = np.array([[a[it]],[b[it]],[c[it]]])
        T_p = 2.0/3.0*np.array([[ np.cos(theta), np.cos(theta-2.0/3.0*np.pi), np.cos(theta+2.0/3.0*np.pi)],
                                [-np.sin(theta),-np.sin(theta-2.0/3.0*np.pi),-np.sin(theta+2.0/3.0*np.pi)]])

        dq=T_p@abc;
        
        d[it] = dq[0]
        q[it] = dq[1]
    
    return d,q

In [5]:
times_sim = data['stru']['time'][0][0][:,0]-0.04-0.001129729-0.0005
i_sa_sim =  data['stru']['i_sabc_1'][0][0][:,0]
i_sb_sim =  data['stru']['i_sabc_2'][0][0][:,0]
i_sc_sim =  data['stru']['i_sabc_3'][0][0][:,0]
i_sd_sim,i_sq_sim = abc2dq(times_sim,i_sa_sim,i_sb_sim,i_sc_sim,omega=2*np.pi*50,theta_0=0.0+np.pi/2)

times_exp = df_typhoon.Time.values
i_sa_exp = df_typhoon.Ia.values
i_sb_exp = df_typhoon.Ib.values
i_sc_exp = df_typhoon.Ic.values
i_sd_exp,i_sq_exp = abc2dq(times_exp,i_sa_exp,i_sb_exp,i_sc_exp,omega=2*np.pi*50,theta_0=0.4+np.pi/2)

In [6]:
i_sa_sim

array([ 0.00000000e+00,  0.00000000e+00, -9.51859239e-05, ...,
        6.39291035e-01,  5.78075870e-01,  5.15042315e-01])

In [11]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), sharex=True)

t_offset = 0.016+0.16+200e-6
scale_amp = 0.9486
#axes[0].plot(times,i_sa)
#axes[0].plot(times,i_sb)
#axes[0].plot(times,i_sc)
axes = [axes]
axes[0].plot(times_sim,i_sd_sim, label='simulink: $i_{sd}$')
axes[0].plot(times_sim,i_sq_sim, label='simulink: $i_{sq}$')
axes[0].plot(times_exp,i_sd_exp, label='typhoon: $i_{sd}$')
axes[0].plot(times_exp,i_sq_exp, label='typhoon: $i_{sq}$')


for ax in axes:
    ax.grid()
    ax.legend()
ax.set_xlabel('Time (s)')
ax.set_xlim([-0.01,0.03])
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
0.000298771-0.0014285

-0.001129729